In [ ]:
import datetime
import pandas as pd

import matplotlib.pyplot as plt
%matplotlib inline
plt.style.use('dark_background')


In [ ]:
# https://www.worldometers.info/world-population/population-by-country/
list_countries = []
list_countries += [('Portugal', 10196709), ('Spain', 46754778), ('France', 65273511), ('Italy', 60461826), ('Germany', 83783942), ('United Kingdom', 67886011), ('Belgium', 11589623), ('Netherlands', 17134872), ('Sweden', 10099265)]
list_countries += [('Ireland', 4937786), ('Luxembourg', 625978), ('Slovakia', 5459642), ('Denmark', 5792202), ('Norway', 5421241), ('Czechia', 10708981), ]
list_countries += [('US', 331002651), ('Brazil', 212559417)]
#list_countries += [('India', 1380004385)]
#list_countries += [('China', 1439323776)]


In [ ]:
!curl --output '/tmp/confirmed.csv' "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_confirmed_global.csv"
!curl --output '/tmp/deaths.csv' "https://raw.githubusercontent.com/CSSEGISandData/COVID-19/master/csse_covid_19_data/csse_covid_19_time_series/time_series_covid19_deaths_global.csv"


In [ ]:
csv = {
    "confirmed": pd.read_csv('/tmp/confirmed.csv'),
    "deaths": pd.read_csv('/tmp/deaths.csv'),
}


In [ ]:
df = {}
for i in csv.keys():
  dates_columns = [col for col in csv[i].columns if '/2' in col]
  df[i] = csv[i].melt(id_vars = ['Country/Region'], value_vars = dates_columns, var_name = 'Date', value_name = 'Confirmed')
  df[i]['Date'] = pd.to_datetime(df[i]['Date'])

  curDate = df[i][df[i]['Country/Region'] == 'Portugal'].tail(1)['Date']

  df[i] = df[i][['Date', 'Country/Region', 'Confirmed']].sort_values(by = 'Date')

  idx = df[i].groupby(['Country/Region', 'Date'])['Confirmed'].transform(max) == df[i]['Confirmed'] 
  df[i] = df[i][idx]
  df[i] = df[i][df[i].Confirmed > 0]
  df[i]['Day'] = df[i].groupby('Country/Region', as_index = False).cumcount()

  df[i].set_index('Date', inplace=True)


In [ ]:
df['confirmed']


In [ ]:
df['deaths']


In [ ]:
((df['deaths']['Country/Region'] == 'Brazil') & (df['deaths']['Confirmed'] > 1000)).sum()


In [ ]:
pt_br = [x for x in list_countries if x[0] in ['Portugal', 'Brazil']]
pt_br, pt_br[1][1] / pt_br[0][1], 1000 / (pt_br[1][1] / pt_br[0][1])


In [ ]:
((df['deaths']['Country/Region'] == 'Portugal') & (df['deaths']['Confirmed'] > 48)).sum()

In [ ]:
br = int(df['deaths'][ df['deaths']['Country/Region'] == 'Brazil' ].tail(1)['Confirmed'])
br, br * 100.0 / pt_br[1][1]


In [ ]:
pt = int(df['deaths'][ df['deaths']['Country/Region'] == 'Portugal' ].tail(1)['Confirmed'])
pt, pt * 100.0 / pt_br[0][1]


In [ ]:
dfb = df['deaths'][ df['deaths']['Country/Region'] == 'Brazil' ].copy()
dfb['Confirmed7'] = dfb['Confirmed'].diff(7).div(7)
dfb['Confirmed1'] = dfb['Confirmed'].diff(1).div(1)
dfb = dfb[ dfb['Confirmed7'].notnull() ]
#dfb
#dfb[['Confirmed7', 'Confirmed1']].plot(figsize=(20,4))
fig, axes = plt.subplots(figsize=(20,4))
axes.plot(dfb.Confirmed1, label="Deaths daily")
axes.plot(dfb.Confirmed7, label="Deaths 7d avg", lw=4)
axes.set_title(f"Brasil Deaths {round(dfb['Confirmed7'][-1])}")
plt.show()


In [ ]:
dfb.tail(2)


In [ ]:
#'Portugal', 10196709), 
#'Brazil', 212559417)]

rel = 10196709 / 212559417
print(300 / rel)